In [1]:
import pulp
import pandas as pd

In [2]:
data = pd.read_excel('./data.xlsx')
data

,Unnamed: 0,地块1,地块2,地块3,地块4,地块5,地块6,计划播种面积（亩）
0,小麦,500,550,630,1000,800,700,76.0
1,玉米,800,700,600,950,900,930,88.0
2,水果,1000,960,840,650,600,700,96.0
3,蔬菜,1200,1040,980,860,880,780,40.0
4,地块面积（亩）,42,56,44,39,60,59,NaN


In [3]:
# 目标函数的系数
z = data.loc[0:3, '地块1':'地块6'].values
z

array([[ 500,  550,  630, 1000,  800,  700],
       [ 800,  700,  600,  950,  900,  930],
       [1000,  960,  840,  650,  600,  700],
       [1200, 1040,  980,  860,  880,  780]])

In [4]:
# 约束条件
a = np.ones((4, 6))
b = data.iloc[:4, -1].tolist() # 播种面积
b

[76.0, 88.0, 96.0, 40.0]

In [5]:
c = data.iloc[-1, 1:7].tolist() # 地块面积
c

[42, 56, 44, 39, 60, 59]

In [6]:
# 定义变量
x = np.array([[pulp.LpVariable(f'x_{i}{j}', lowBound=0) for j in range(1, 7)] for i in range(1, 5)])
x

array([[x_11, x_12, x_13, x_14, x_15, x_16],
       [x_21, x_22, x_23, x_24, x_25, x_26],
       [x_31, x_32, x_33, x_34, x_35, x_36],
       [x_41, x_42, x_43, x_44, x_45, x_46]], dtype=object)

In [7]:
#确定最大化最小化问题
m = pulp.LpProblem(sense=pulp.LpMaximize)

In [8]:
#定义目标函数，lpDot可以将两个列表的对应位相乘再加和
m += pulp.lpDot(z, x)
print(pulp.lpDot(z, x))

500*x_11 + 550*x_12 + 630*x_13 + 1000*x_14 + 800*x_15 + 700*x_16 + 800*x_21 + 700*x_22 + 600*x_23 + 950*x_24 + 900*x_25 + 930*x_26 + 1000*x_31 + 960*x_32 + 840*x_33 + 650*x_34 + 600*x_35 + 700*x_36 + 1200*x_41 + 1040*x_42 + 980*x_43 + 860*x_44 + 880*x_45 + 780*x_46


In [9]:
# 设置约束条件
# 播种面积
for i in range(4):
    m += (pulp.lpDot(a[i], x[i]) >= b[i])
    print(pulp.lpDot(a[i], x[i]) >= b[i])

x_11 + x_12 + x_13 + x_14 + x_15 + x_16 >= 76.0
x_21 + x_22 + x_23 + x_24 + x_25 + x_26 >= 88.0
x_31 + x_32 + x_33 + x_34 + x_35 + x_36 >= 96.0
x_41 + x_42 + x_43 + x_44 + x_45 + x_46 >= 40.0


In [10]:
# 地块面积
for j in range(6):
    m += (pulp.lpSum(x[:, j]) <= c[j])
    print(pulp.lpSum(x[:, j]) <= c[j])

x_11 + x_21 + x_31 + x_41 <= 42
x_12 + x_22 + x_32 + x_42 <= 56
x_13 + x_23 + x_33 + x_43 <= 44
x_14 + x_24 + x_34 + x_44 <= 39
x_15 + x_25 + x_35 + x_45 <= 60
x_16 + x_26 + x_36 + x_46 <= 59


In [17]:
m.solve() #求解

#输出结果
print(f'优化结果：{pulp.value(m.objective)}')

优化结果：284230.0


In [18]:
print('参数取值：')
pd.DataFrame([[pulp.value(var) for var in ls] for ls in x], index=data.iloc[:4, 0].values, columns=data.columns[1:7])

参数取值：


,地块1,地块2,地块3,地块4,地块5,地块6
小麦,0.0,0.0,6.0,39.0,31.0,0.0
玉米,0.0,0.0,0.0,0.0,29.0,59.0
水果,2.0,56.0,38.0,0.0,0.0,0.0
蔬菜,40.0,0.0,0.0,0.0,0.0,0.0
